In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle
import json

import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 500)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)




module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from retrieval_algorithms import BM25RetrievalAlgorithm
from retrieval_algorithms.sent2vec_retrieval_algorithm import Sent2VecRetrievalAlgorithm

from evaluation import *
import evaluation
from manual_expert_list import manual_expert_list


from expert_voting.expert_voting import ExpertVoting
from expert_voting.scored_document import ScoredDocument

from expert_voting.document_score_modifiers.paper_age_modifier import PaperAgeModifier
from expert_voting.document_score_modifiers.citation_count_modifier import CitationCountModifier
from expert_voting.document_score_modifiers.exponential_modifier import ExponentialModifier
from expert_voting.document_score_modifiers.reciprocal_rank_modifier import ReciprocalRankModifier

from expert_voting.document_score_aggregations.summation_aggregation import SummationAggregation
from expert_voting.document_score_aggregations.uniform_aggregation import UniformAggregation
from expert_voting.document_score_aggregations.author_order_aggregation import AuthorOrderAggregation

from expert_voting.author_score_modifiers.paper_count_modifier import PaperCountModifier
from expert_voting.author_score_modifiers.co_author_modifier import CoAuthorModifier
from expert_voting.author_score_modifiers.last_kit_paper_modifier import LastKitPaperModifier
from expert_voting.author_score_modifiers.kit_paper_ratio_modifier import KitPaperRatioModifier


from preprocessing import Corpus, BasicPreprocessing, BigramPreprocessor, SpacyPreprocessor, StopWordPreprocessor
from tqdm.notebook import tqdm
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tqdm.pandas()

In [2]:
test_data = ("manual test data", [{"keyword": topic, "paper_ids":experts} for topic, experts  in manual_expert_list.items()])

In [3]:
base_file =  "../../data/kit_expert_2019_all_papers.csv"

p = [BasicPreprocessing(), StopWordPreprocessor()]
papers_basic = Corpus(base_file, p)

p = [BasicPreprocessing(), StopWordPreprocessor(), SpacyPreprocessor(lemmatization="all")]
papers_basic_lemmatization_all = Corpus(base_file, p, load_from_cache=True, n_jobs=16)

p = [BasicPreprocessing(), StopWordPreprocessor(), SpacyPreprocessor(lemmatization="nouns")]
papers_basic_lemmatization_nouns = Corpus(base_file, p, load_from_cache=True, n_jobs=16)

In [4]:
# file_path = "../../data/models/sent2vec/sent2vec_wiki_bigrams.model"
# file_path = "../../data/models/tfidf/bm25_bigram_k1=0.4_b=0.8.model"
file_path = "../../data/models/ensemble_model.model"
with open(file_path, "rb") as file:
    model = pickle.load(file)
#     pickle.dump(model, file)

In [5]:
model.weight = 0.04

In [6]:
with open("../../data/kit_expert_2019_all_paper_info.json") as file:
    paper_info = json.load(file)

In [7]:
with open("../../data/kit_expert_2019_all_author_info.json") as file:
    author_info = json.load(file)

In [8]:
def t_test(mean1, var1, mean2, var2):
    return (mean2-mean1)/np.sqrt((var1+var2)/12)

In [9]:
def t_test_table(df, reference):
    df = df.sort_index(axis = 1)
    for i in set(df.columns.droplevel(2)):
        df[(*i,"t-score")] = t_test(reference[i]["avg"], reference[i]["var"],df[i]["avg"],  df[i]["var"])
    return df.sort_index(axis = 1)

In [380]:
boost_top_papers_models = [
    ("Simple Baseline", ExpertVoting([], SummationAggregation(paper_info), [], model), None),
    ("Exponential mod.", ExpertVoting([ExponentialModifier()], SummationAggregation(paper_info), [], model), None),
    ("Reciprocal rank mod.", ExpertVoting([ReciprocalRankModifier(False)], SummationAggregation(paper_info), [], model), None),
    ("Reciprocal rank score mod.", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info), [], model), None),
    ("Paper count mod.", ExpertVoting([], SummationAggregation(paper_info), [PaperCountModifier(paper_info)], model), None),
    ("Top 10 papers aggregation", ExpertVoting([], SummationAggregation(paper_info,10), [], model), None),
    ("RR + Top 10 combinded", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info,10), [], model), None),
]
boost_top_papers_result = evaluate_models(boost_top_papers_models, [test_data], n_jobs=1)

In [382]:
print(author_ranking_to_latex(boost_top_papers_result))
t_test_table(boost_top_papers_result,boost_top_papers_result.loc["Simple Baseline"])

Simple Baseline & 0.354 & 0.333 & 0.398 & 0.295 & 0.265 & 0.242\\
Exponential mod. & 0.565 & 0.489 & 0.498 & 0.406 & 0.377 & 0.347\\
Reciprocal rank mod. & 0.768 & 0.743 & 0.799 & 0.597 & 0.652 & 0.590\\
Reciprocal rank score mod. & 0.801 & 0.784 & 0.812 & 0.660 & 0.686 & 0.631\\
Paper count mod. & 0.404 & 0.409 & 0.473 & 0.325 & 0.330 & 0.268\\
Top 10 papers aggregation & 0.785 & 0.685 & 0.647 & 0.555 & 0.574 & 0.534\\
RR + Top 10 combinded & 0.818 & 0.803 & 0.830 & 0.658 & 0.703 & 0.657\\



manual test data                                                                                                                                         
                                     R-prec                   bpref                     mAP                    p@10                    p@20                     p@5                 
                                        avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var
Simple Baseline                       0.295  0.000000  0.023  0.242  0.000000  0.019  0.265  0.000000  0.023  0.333  0.000000  0.033  0.398  0.000000  0.021  0.354  0.000000  0.051
Exponential mod.                      0.406  1.624874  0.033  0.347  1.510309  0.039  0.377  1.487833  0.045  0.489  1.986549  0.041  0.498  1.256562  0.055  0.565  1.926158  0.093
Reciprocal rank mod.                  0.597  5.941782  0.008  0.590  6.353582  0.017  0.652  6.703037  0.017  0.743  6.001785  0.023  0.799  7.034005  0.018  0.768  4.728213  0.041
Reciprocal rank score mod.            0.660  6.857156  0.011  0.631  7.202880  0.016  0.686  7.291934  0.017  0.784  6.431928  0.026  0.812  7.262040  0.018  0.801  5.544353  0.027
Paper count mod.                      0.325  0.455733  0.029  0.268  0.439480  0.023  0.330  1.006976  0.027  0.409  1.154523  0.019  0.473  1.069611  0.038  0.404  0.495885  0.071
Top 10 papers aggregation             0.555  3.988218  0.028  0.534  4.435798  0.033  0.574  4.564232  0.032  0.685  4.642039  0.036  0.647  3.551109  0.038  0.785  4.652103  0.052
RR + Top 10 combinded                 0.658  6.450679  0.015  0.657  7.684307  0.016  0.703  7.586383  0.017  0.803  6.760433  0.025  0.830  7.302133  0.021  0.818  5.613086  0.031

In [383]:
aggregation_models = [
    ("Baseline", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [], model), None),
    ("RR uni", ExpertVoting([ReciprocalRankModifier(True)], UniformAggregation(paper_info, 10), [], model), None),
    ("RR author order 0.3", ExpertVoting([ReciprocalRankModifier(True)], AuthorOrderAggregation(paper_info,0.2, 10), [], model), None),
    ("RR author order 0.35", ExpertVoting([ReciprocalRankModifier(True)], AuthorOrderAggregation(paper_info,0.35, 10), [], model), None),
    ("RR author order 0.4", ExpertVoting([ReciprocalRankModifier(True)], AuthorOrderAggregation(paper_info,0.4, 10), [], model), None),
    ("RR author order 0.45", ExpertVoting([ReciprocalRankModifier(True)], AuthorOrderAggregation(paper_info,0.45, 10), [], model), None),
    ("RR author order 0.5", ExpertVoting([ReciprocalRankModifier(True)], AuthorOrderAggregation(paper_info,0.5, 10), [], model), None),
    ("RR author order 0.55", ExpertVoting([ReciprocalRankModifier(True)], AuthorOrderAggregation(paper_info,0.55, 10), [], model), None),
]
aggregation_result = evaluate_models(aggregation_models, [test_data], n_jobs=1)


In [384]:
print(author_ranking_to_latex(aggregation_result))
t_test_table(aggregation_result,aggregation_result.loc["Baseline"])

Baseline & 0.818 & 0.803 & 0.830 & 0.658 & 0.703 & 0.657\\
RR uni & 0.764 & 0.767 & 0.809 & 0.591 & 0.650 & 0.592\\
RR author order 0.3 & 0.758 & 0.704 & 0.778 & 0.613 & 0.640 & 0.609\\
RR author order 0.35 & 0.775 & 0.791 & 0.805 & 0.645 & 0.677 & 0.643\\
RR author order 0.4 & 0.792 & 0.811 & 0.830 & 0.646 & 0.693 & 0.654\\
RR author order 0.45 & 0.792 & 0.816 & 0.830 & 0.644 & 0.693 & 0.653\\
RR author order 0.5 & 0.775 & 0.837 & 0.830 & 0.644 & 0.693 & 0.653\\
RR author order 0.55 & 0.775 & 0.815 & 0.825 & 0.608 & 0.686 & 0.637\\



manual test data                                                                                                                                         
                               R-prec                   bpref                     mAP                    p@10                    p@20                     p@5                 
                                  avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var
Baseline                        0.658  0.000000  0.015  0.657  0.000000  0.016  0.703  0.000000  0.017  0.803  0.000000  0.025  0.830  0.000000  0.021  0.818  0.000000  0.031
RR uni                          0.591 -1.498166  0.009  0.592 -1.278860  0.015  0.650 -1.026341  0.015  0.767 -0.536656  0.029  0.809 -0.346804  0.023  0.764 -0.757390  0.030
RR author order 0.3             0.613 -0.799671  0.023  0.609 -0.758947  0.032  0.640 -0.947967  0.036  0.704 -1.278085  0.047  0.778 -0.813759  0.028  0.758 -0.785584  0.039
RR author order 0.35            0.645 -0.274064  0.012  0.643 -0.239512  0.025  0.677 -0.434340  0.026  0.791 -0.195959  0.020  0.805 -0.412861  0.023  0.775 -0.571222  0.037
RR author order 0.4             0.646 -0.248424  0.013  0.654 -0.051962  0.024  0.693 -0.173205  0.023  0.811  0.125194  0.024  0.830  0.000000  0.024  0.792 -0.353270  0.034
RR author order 0.45            0.644 -0.284787  0.014  0.653 -0.067612  0.026  0.693 -0.173205  0.023  0.816  0.207723  0.022  0.830  0.000000  0.024  0.792 -0.353270  0.034
RR author order 0.5             0.644 -0.284787  0.014  0.653 -0.067612  0.026  0.693 -0.175412  0.022  0.837  0.532073  0.024  0.830  0.000000  0.024  0.775 -0.571222  0.037
RR author order 0.55            0.608 -0.835269  0.028  0.637 -0.330289  0.028  0.686 -0.302098  0.021  0.815  0.198173  0.019  0.825 -0.079894  0.026  0.775 -0.571222  0.037

## Paper Metadata

In [ ]:
search_paperage_models = [
    (f"PaperAgeModifier {i}, {j}", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,i,j)], SummationAggregation(paper_info, 10), [], model), None)
    for i in range(1,11) for j in np.arange(0,1.1,0.1)
]
search_paperage_result = evaluate_models(search_paperage_models, [test_data], n_jobs=5)
search_paperage_result.sort_values(by=("manual test data","mAP","avg"), ascending=False).head(5)

In [393]:
search_paperage_result.sort_values(by=("manual test data","mAP","avg"), ascending=False).head(5)

manual test data                                                                             
                                                     p@5          p@10          p@20        R-prec           mAP         bpref       
                                                     avg    var    avg    var    avg    var    avg    var    avg    var    avg    var
PaperAgeModifier 3, 0.6000000000000001             0.818  0.031  0.812  0.020  0.826  0.023  0.680  0.010  0.710  0.017  0.661  0.016
PaperAgeModifier 3, 0.30000000000000004            0.818  0.031  0.803  0.023  0.826  0.023  0.685  0.012  0.710  0.018  0.662  0.016
PaperAgeModifier 3, 0.2                            0.801  0.027  0.817  0.026  0.826  0.023  0.695  0.013  0.710  0.020  0.664  0.018
PaperAgeModifier 2, 0.6000000000000001             0.818  0.031  0.812  0.020  0.826  0.023  0.680  0.010  0.710  0.017  0.663  0.016
PaperAgeModifier 3, 0.4                            0.818  0.031  0.812  0.020  0.826  0.023  0.685  0.012  0.709  0.018  0.661  0.017

In [387]:
search_citationcount_models = [
    (f"Citation Count {i}", ExpertVoting([ReciprocalRankModifier(True), CitationCountModifier(paper_info,i)], SummationAggregation(paper_info, 10), [], model), None)
    for i in [1] 
]
search_citationcount_result = evaluate_models(search_citationcount_models, [test_data], n_jobs=1)

In [388]:
search_citationcount_result

manual test data                                                                             
                              p@5          p@10          p@20        R-prec           mAP         bpref       
                              avg    var    avg    var    avg    var    avg    var    avg    var    avg    var
Citation Count 1            0.768  0.019  0.789  0.019  0.802  0.022   0.63  0.011  0.673  0.016  0.623  0.017

In [ ]:
search_coauthor_models = [
    (f"CoAuthor {i}", ExpertVoting([ReciprocalRankModifier(True), ], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,i)], model), None)
    for i in [0.01,0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018]#[0.001,0.002,0.003,0.006,0.008,0.010] 
]
search_coauthor_result = evaluate_models(search_coauthor_models, [test_data], n_jobs=1)

In [390]:
search_coauthor_result

manual test data                                                                             
                            p@5          p@10          p@20        R-prec           mAP         bpref       
                            avg    var    avg    var    avg    var    avg    var    avg    var    avg    var
CoAuthor 0.01             0.818  0.031  0.782  0.021  0.837  0.018  0.658  0.015  0.711  0.016  0.659  0.016
CoAuthor 0.011            0.818  0.031  0.782  0.021  0.837  0.018  0.658  0.015  0.713  0.016  0.659  0.016
CoAuthor 0.012            0.818  0.031  0.782  0.021  0.837  0.018  0.658  0.015  0.713  0.016  0.659  0.016
CoAuthor 0.013            0.818  0.031  0.782  0.021  0.837  0.018  0.658  0.015  0.713  0.016  0.660  0.016
CoAuthor 0.014            0.818  0.031  0.782  0.021  0.837  0.018  0.658  0.015  0.713  0.016  0.660  0.016
CoAuthor 0.015            0.818  0.031  0.782  0.021  0.837  0.018  0.658  0.015  0.713  0.016  0.660  0.016
CoAuthor 0.016            0.818  0.031  0.768  0.022  0.837  0.018  0.658  0.015  0.712  0.016  0.657  0.017
CoAuthor 0.017            0.818  0.031  0.768  0.022  0.837  0.018  0.658  0.015  0.711  0.016  0.656  0.017
CoAuthor 0.018            0.818  0.031  0.768  0.022  0.837  0.018  0.658  0.015  0.711  0.016  0.657  0.017

In [391]:
metadata_models = [
    (f"CoAuthor {0.015}", ExpertVoting([ReciprocalRankModifier(True), ], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015)], model), None),
    (f"Citation Count", ExpertVoting([ReciprocalRankModifier(True), CitationCountModifier(paper_info,1)], SummationAggregation(paper_info, 10), [], model), None),
    (f"PaperAgeModifier {2}, {0.6}", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [], model), None),
    (f"PaperAgeModifier + CoAuthor", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015)], model), None),
]
metadata_results = evaluate_models(metadata_models, [test_data], n_jobs=1)

In [395]:
print(author_ranking_to_latex(metadata_results))
t_test_table(metadata_results,aggregation_result.loc["Baseline"])

CoAuthor 0.015 & 0.818 & 0.782 & 0.837 & 0.658 & 0.713 & 0.660\\
Citation Count & 0.768 & 0.789 & 0.802 & 0.630 & 0.673 & 0.623\\
PaperAgeModifier 2, 0.6 & 0.818 & 0.812 & 0.826 & 0.680 & 0.710 & 0.663\\
PaperAgeModifier + CoAuthor & 0.818 & 0.791 & 0.833 & 0.664 & 0.715 & 0.663\\



manual test data                                                                                                                                         
                                      R-prec                   bpref                     mAP                    p@10                    p@20                     p@5                 
                                         avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var
CoAuthor 0.015                         0.658  0.000000  0.015  0.660  0.058095  0.016  0.713  0.190693  0.016  0.782 -0.339181  0.021  0.837  0.122788  0.018  0.818  0.000000  0.031
Citation Count                         0.630 -0.601536  0.011  0.623 -0.648355  0.017  0.673 -0.572078  0.016  0.789 -0.231202  0.019  0.802 -0.467751  0.022  0.768 -0.774597  0.019
PaperAgeModifier 2, 0.6                0.680  0.481996  0.010  0.663  0.116190  0.016  0.710  0.131507  0.017  0.812  0.146969  0.020  0.826 -0.066058  0.023  0.818  0.000000  0.031
PaperAgeModifier + CoAuthor            0.664  0.118049  0.016  0.663  0.116190  0.016  0.715  0.228831  0.016  0.791 -0.202837  0.017  0.833  0.051324  0.020  0.818  0.000000  0.031

## Candidate list errors

### KIT ratio modifier

In [ ]:
search_kitratio_models = [
    (f"KitRatio {i}", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info,10), [KitPaperRatioModifier(author_info, i)], model), None)
    for i in [0.0001,0.001,0.01,0.1,0.2,0.5,1,2,3,4,5,6]
]
search_kitratio_results = evaluate_models(search_kitratio_models, [test_data], n_jobs=1)

In [354]:
search_kitratio_results

manual test data                                                                             
                             p@5          p@10          p@20        R-prec           mAP         bpref       
                             avg    var    avg    var    avg    var    avg    var    avg    var    avg    var
KitRatio 0.0001            0.862  0.025  0.812  0.020  0.838  0.025  0.664  0.016  0.720  0.017  0.671  0.019
KitRatio 0.001             0.862  0.025  0.812  0.020  0.838  0.025  0.664  0.016  0.720  0.017  0.671  0.019
KitRatio 0.01              0.862  0.025  0.812  0.020  0.838  0.025  0.664  0.016  0.720  0.017  0.671  0.019
KitRatio 0.1               0.862  0.025  0.812  0.020  0.838  0.025  0.664  0.016  0.720  0.017  0.671  0.019
KitRatio 0.2               0.862  0.025  0.812  0.020  0.838  0.025  0.664  0.016  0.718  0.017  0.670  0.018
KitRatio 0.5               0.862  0.025  0.812  0.020  0.842  0.023  0.664  0.016  0.716  0.016  0.669  0.018
KitRatio 1                 0.862  0.025  0.812  0.020  0.842  0.023  0.664  0.016  0.712  0.017  0.668  0.018
KitRatio 2                 0.862  0.025  0.812  0.020  0.842  0.023  0.666  0.013  0.713  0.016  0.670  0.018
KitRatio 3                 0.862  0.025  0.820  0.019  0.842  0.023  0.666  0.013  0.714  0.016  0.670  0.017
KitRatio 4                 0.862  0.025  0.820  0.019  0.842  0.023  0.666  0.013  0.715  0.016  0.673  0.017
KitRatio 5                 0.862  0.025  0.820  0.019  0.842  0.023  0.666  0.013  0.714  0.016  0.671  0.017
KitRatio 6                 0.835  0.026  0.820  0.019  0.842  0.023  0.666  0.013  0.712  0.015  0.662  0.016

### Last KIT paper modifier

In [403]:
search_lastkitpaper_models = [
    (f"LastKitPaper {i}, {j}", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [LastKitPaperModifier(author_info, i, j)], model), None)
    for i in range(1,11) for j in np.arange(0,1.1,0.1)
]

In [ ]:
search_lastkitpaper_result = evaluate_models(search_lastkitpaper_models, [test_data], n_jobs=5)

In [405]:
search_lastkitpaper_result.sort_values(by=("manual test data","mAP","avg"), ascending=False).head(5)

manual test data                                                                            
                                  p@5          p@10         p@20        R-prec           mAP         bpref       
                                  avg    var    avg    var   avg    var    avg    var    avg    var    avg    var
LastKitPaper 3, 0.1             0.863  0.025  0.798  0.022  0.82  0.022  0.666  0.013  0.707  0.018  0.665  0.016
LastKitPaper 9, 0.2             0.863  0.025  0.798  0.022  0.82  0.022  0.661  0.012  0.707  0.016  0.657  0.017
LastKitPaper 4, 0.2             0.863  0.025  0.798  0.022  0.82  0.022  0.666  0.013  0.707  0.018  0.666  0.017
LastKitPaper 5, 0.2             0.863  0.025  0.798  0.022  0.82  0.022  0.661  0.012  0.705  0.018  0.664  0.016
LastKitPaper 10, 0.2            0.863  0.025  0.798  0.022  0.82  0.022  0.661  0.012  0.705  0.016  0.657  0.017

In [408]:
search_non_kit_author_models = [
    ("Baseline", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [], model), None),
    ("LastKitPaper 3, 0.1", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [LastKitPaperModifier(author_info, 3, 0.1)], model), None),
    ("KitRatio 0.01", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [ KitPaperRatioModifier(author_info, 0.01)], model), None),
    ("KitRatio 0.01 + LastKitPaper 3, 0.1", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [KitPaperRatioModifier(author_info, 0.01), LastKitPaperModifier(author_info, 3, 0.1)], model), None),
    (f"All", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015),KitPaperRatioModifier(author_info, 0.01), LastKitPaperModifier(author_info, 3, 0.1)], model), None),
]
search_non_kit_author_result = evaluate_models(search_non_kit_author_models, [test_data], n_jobs=1)

In [409]:
print(author_ranking_to_latex(search_non_kit_author_result))
t_test_table(search_non_kit_author_result,aggregation_result.loc["Baseline"])

Baseline & 0.818 & 0.803 & 0.830 & 0.658 & 0.703 & 0.657\\
KitRatio 0.01 & 0.862 & 0.812 & 0.838 & 0.664 & 0.720 & 0.671\\
LastKitPaper 3, 0.1 & 0.863 & 0.798 & 0.820 & 0.666 & 0.707 & 0.665\\
KitRatio 0.01 + LastKitPaper 3, 0.1 & 0.863 & 0.812 & 0.807 & 0.661 & 0.723 & 0.671\\
All & 0.879 & 0.798 & 0.807 & 0.673 & 0.730 & 0.676\\



manual test data                                                                                                                                         
                                              R-prec                   bpref                     mAP                    p@10                    p@20                     p@5                 
                                                 avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var
Baseline                                       0.658  0.000000  0.015  0.657  0.000000  0.016  0.703  0.000000  0.017  0.803  0.000000  0.025  0.830  0.000000  0.021  0.818  0.000000  0.031
KitRatio 0.01                                  0.664  0.118049  0.016  0.671  0.259230  0.019  0.720  0.319374  0.017  0.812  0.146969  0.020  0.838  0.129212  0.025  0.862  0.644094  0.025
LastKitPaper 3, 0.1                            0.666  0.165616  0.013  0.665  0.154919  0.016  0.707  0.074066  0.018  0.798 -0.079894  0.022  0.820 -0.167054  0.022  0.863  0.658733  0.025
KitRatio 0.01 + LastKitPaper 3, 0.1            0.661  0.060000  0.015  0.671  0.263014  0.018  0.723  0.375735  0.017  0.812  0.146969  0.020  0.807 -0.363662  0.027  0.863  0.621059  0.032
All                                            0.673  0.305129  0.014  0.676  0.356948  0.018  0.730  0.507242  0.017  0.798 -0.079894  0.022  0.807 -0.363662  0.027  0.879  0.945008  0.019

In [411]:
removed_non_kit_experts_models = [
    ("removed non kit candidates", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info,10), [], model), None),
    (f"metadata", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015)], model), None),
    (f"All", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015),KitPaperRatioModifier(author_info, 0.01), LastKitPaperModifier(author_info, 3, 0.1)], model), None),

]
evaluation.ignored_ids = [
    # IoT
    2228585409,
    2085145230,
    2227135787,
    2226123400,
    2228992999,
    954006143,
    # blockchain
    2948008269,
    1808354562,
    # nlp
    2050175453,
    2045275999,
    2230575466,
    2294519613,
    2254179337,
    2170753228,
    # autonomous driving
    2231714904,
    2978751607,
    2706472602,
    2116123198,
    2563466229,
    # humanoid
    # evolutionary algorithm
    2592252759,
    2120798276,
    2085145230,
    # remote sensing
    1882366596,
    1905280943,
    1966820645,
    2226192829,
    # action recognition
    2105785781,
    2105785781,
    39126808,
    # cellular automaton
    # electronic voting
    2147598239,
    2218633584,
    # kalman filter	
    2147368211,
    2088092445,
    #software architecture
    2892873526,
    2224933589
]
# evaluation.ignored_ids = []
removed_non_kit_experts_result = evaluate_models(removed_non_kit_experts_models, [test_data], n_jobs=1)
evaluation.ignored_ids = []

In [445]:
print(author_ranking_to_latexcombined_result_non_kit_experts_result))
t_test_table(removed_non_kit_experts_result,search_non_kit_author_result.loc["Baseline"])

removed non kit candidates & 0.872 & 0.866 & 0.887 & 0.701 & 0.791 & 0.739\\
metadata & 0.851 & 0.866 & 0.887 & 0.709 & 0.792 & 0.730\\
All & 0.896 & 0.866 & 0.877 & 0.716 & 0.794 & 0.726\\



manual test data                                                                                                                                         
                                     R-prec                   bpref                     mAP                    p@10                    p@20                     p@5                 
                                        avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var
removed non kit candidates            0.701  0.982189  0.008  0.739  1.697561  0.012  0.791  1.927983  0.008  0.866  1.006659  0.022  0.887  0.987269  0.019  0.872  0.763675  0.029
metadata                              0.709  1.164922  0.008  0.730  1.632330  0.008  0.792  2.032903  0.006  0.866  1.028786  0.020  0.887  0.987269  0.019  0.851  0.470629  0.028
All                                   0.716  1.246040  0.011  0.726  1.380000  0.014  0.794  1.918448  0.010  0.866  1.052439  0.018  0.877  0.776179  0.023  0.896  1.208371  0.019

In [11]:
combined_models = [
    ("Baseline", ExpertVoting([ReciprocalRankModifier(True)], SummationAggregation(paper_info, 10), [], model), None),
    ("PaperAgeModifier", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [], model), None),
    ("PaperAge + CoAuthor", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015)], model), None),
    ("PaperAge + CoAuthor + LastKitPaper", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015), LastKitPaperModifier(author_info, 3, 0.1)], model), None),
    ("PaperAge + CoAuthor + LastKitPaper + KitRatio", ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015), LastKitPaperModifier(author_info, 3, 0.1),KitPaperRatioModifier(author_info, 0.01)], model), None),
]
combined_result = evaluate_models(combined_models, [test_data], n_jobs=1)

In [12]:
print(author_ranking_to_latex(combined_result))
t_test_table(combined_result,combined_result.loc["Baseline"])

Baseline & 0.818 & 0.803 & 0.830 & 0.658 & 0.703 & 0.657\\
PaperAgeModifier & 0.818 & 0.812 & 0.826 & 0.680 & 0.710 & 0.663\\
PaperAge + CoAuthor & 0.818 & 0.791 & 0.833 & 0.664 & 0.715 & 0.663\\
PaperAge + CoAuthor + LastKitPaper & 0.863 & 0.777 & 0.818 & 0.685 & 0.714 & 0.671\\
PaperAge + CoAuthor + LastKitPaper + KitRatio & 0.879 & 0.798 & 0.807 & 0.673 & 0.730 & 0.676\\



manual test data                                                                                                                                         
                                                        R-prec                   bpref                     mAP                    p@10                    p@20                     p@5                 
                                                           avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var    avg   t-score    var
Baseline                                                 0.658  0.000000  0.015  0.657  0.000000  0.016  0.703  0.000000  0.017  0.803  0.000000  0.025  0.830  0.000000  0.021  0.818  0.000000  0.031
PaperAgeModifier                                         0.680  0.481996  0.010  0.663  0.116190  0.016  0.710  0.131507  0.017  0.812  0.146969  0.020  0.826 -0.066058  0.023  0.818  0.000000  0.031
PaperAge + CoAuthor                                      0.664  0.118049  0.016  0.663  0.116190  0.016  0.715  0.228831  0.016  0.791 -0.202837  0.017  0.833  0.051324  0.020  0.818  0.000000  0.031
PaperAge + CoAuthor + LastKitPaper                       0.685  0.569210  0.012  0.671  0.271109  0.016  0.714  0.203680  0.018  0.777 -0.434340  0.018  0.818 -0.198173  0.023  0.863  0.658733  0.025
PaperAge + CoAuthor + LastKitPaper + KitRatio            0.673  0.305129  0.014  0.676  0.356948  0.018  0.730  0.507242  0.017  0.798 -0.079894  0.022  0.807 -0.363662  0.027  0.879  0.945008  0.019

## Kompetenzpool evaluation

In [10]:
from kompetenzpool_expert_list import kompetenzpool_expert_list
kompetenzpool_test_data = ("kompetenzpool test data", [{"keyword": topic, "paper_ids":experts} for topic, experts  in kompetenzpool_expert_list.items()])

In [11]:
best_voting_model = ExpertVoting([ReciprocalRankModifier(True), PaperAgeModifier(paper_info,2,0.6)], SummationAggregation(paper_info, 10), [CoAuthorModifier(paper_info,0.015), LastKitPaperModifier(author_info, 3, 0.1),KitPaperRatioModifier(author_info, 0.01)], model)

In [14]:
metrics = [("r@5",recall_at(5)), ("r@10",recall_at(10)), ("r@20", recall_at(20))]

In [19]:
kompetenzpool_results = evaluate_models([("Best expert voting model",best_voting_model,None)], [kompetenzpool_test_data],1, metrics)

In [16]:
kompetenzpool_results

kompetenzpool test data                                   
                                             r@5          r@10          r@20       
                                             avg    var    avg    var    avg    var
Best expert voting model                   0.521  0.175  0.776  0.076  0.913  0.027

In [17]:
kompetenzpool_results
print(author_ranking_to_latex(kompetenzpool_results))

Best expert voting model & 0.521 & 0.776 & 0.913\\



In [23]:
best_voting_model.get_ranking("natural language understanding t", author_info).head(10)

,id,score,relevant_docs,#rel docs,name
1,2794237920,0.781,"[(2905564348, 1.285), (2802862911, 0.053), (2977544538, 0.008), (2992984162, 0.006), (2992171449, 0.0)]",5,stefan constantin
2,2045275999,0.755,"[(2905564348, 1.285), (2802862911, 0.053), (2095650036, 0.003), (2406079600, 0.002), (2250741237, 0.002), (2293139442, 0.002), (2886847936, 0.002), (2407834842, 0.001), (2403566681, 0.001), (2233968761, 0.001), (2181317029, 0.001), (2402510766, 0.001), (2183405210, 0.001), (2154886093, 0.001), (2770394828, 0.001), (2555745756, 0.001), (2063697624, 0.001), (2126943543, 0.001), (2250751800, 0.001), (2250336789, 0.001), (2915824390, 0.001), (2405083905, 0.001), (2294770573, 0.001), (2251495365,...",44,jan niehues
0,2061074560,0.699,"[(2905564348, 1.285), (188131813, 0.561), (2439063616, 0.21), (2147764898, 0.072), (2511922227, 0.063), (2016244848, 0.054), (2802862911, 0.053), (56456872, 0.027), (2170418652, 0.02), (2006292950, 0.016), (2465729512, 0.014), (2251328660, 0.012), (58629251, 0.011), (2184055828, 0.01), (2501821520, 0.01), (2542201253, 0.01), (2977544538, 0.008), (1579426566, 0.008), (1991635536, 0.007), (2992984162, 0.006), (17129599, 0.006), (2102356979, 0.006), (2058107683, 0.005), (54297365, 0.005), (2938...",418,alex waibel
3,2074083528,0.513,"[(2006989109, 0.381), (2468231206, 0.13), (2999851839, 0.045), (2162272036, 0.037), (2613872948, 0.034), (2002812632, 0.033), (2954456800, 0.03), (2059821724, 0.027), (2023337578, 0.026), (2377424484, 0.024), (2500992586, 0.022), (2903764084, 0.019), (2295724997, 0.012), (2026858563, 0.01), (1964308399, 0.009), (2033488294, 0.008), (2187769469, 0.005), (2188982317, 0.005), (2729209889, 0.004), (2321406831, 0.004), (2921101790, 0.002), (2603705891, 0.002), (2112574403, 0.002), (2596118607, 0....",145,walter f tichy
5,1047662447,0.387,"[(2219532056, 0.285), (2767706754, 0.125), (2982369787, 0.065), (2614519765, 0.035), (1984106238, 0.009), (2562952297, 0.008), (2782973409, 0.006), (2051669146, 0.001), (1967407878, 0.001), (16373103, 0.001), (2784067459, 0.001), (1492952184, 0.001), (2544432894, 0.001), (2912701897, 0.0), (23192490, 0.0), (2123484812, 0.0), (2172323324, 0.0), (2152897124, 0.0), (2793384610, 0.0), (2129157041, 0.0), (1995034212, 0.0), (2657657473, 0.0), (1487977235, 0.0), (2066407408, 0.0), (2103899131, 0.0)...",244,tamim asfour
4,2231152044,0.349,"[(2006989109, 0.381), (2791258916, 0.202), (2377424484, 0.024), (2500992586, 0.022), (2797281399, 0.017), (2729209889, 0.004), (2999019883, 0.0), (2967736183, 0.0)]",8,sebastian weigelt
7,2161631798,0.305,"[(2219532056, 0.285), (2767706754, 0.125), (2782973409, 0.006), (1967407878, 0.001), (1492952184, 0.001), (2912701897, 0.0), (2890646997, 0.0), (2571411619, 0.0), (2981773282, 0.0), (2973609966, 0.0), (2034173299, 0.0), (2004707451, 0.0), (2570226614, 0.0), (2466581433, 0.0), (1509536730, 0.0), (2963573295, 0.0), (2555794269, 0.0), (2780774970, 0.0), (2912479198, 0.0), (2034131903, 0.0), (2783028038, 0.0), (2789941543, 0.0), (2403441342, 0.0)]",23,mirko wachter
6,2578689274,0.240,"[(2791258916, 0.202), (2954447082, 0.195), (2023337578, 0.026), (2797281399, 0.017), (2729209889, 0.004), (2999019883, 0.0)]",6,tobias hey
11,268444295,0.157,"[(2188178817, 0.097), (2051979997, 0.025), (1607986467, 0.021), (1854552892, 0.015), (1598223721, 0.012), (2170495136, 0.011), (772893982, 0.009), (1501979881, 0.008), (2964543620, 0.007), (144732048, 0.007), (407031400, 0.006), (134769278, 0.004), (1983101400, 0.004), (1600230604, 0.003), (2906833531, 0.002), (2919272874, 0.002), (1747247384, 0.002), (2132232128, 0.001), (147727067, 0.001), (2011938319, 0.001), (66839265, 0.001), (2270448588, 0.0), (2809509653, 0.0), (2997921605, 0.0), (294...",183,alexander maedche
8,2294519613,0.136,"[(2468231206, 0.13), (2999851839, 0.045), (2613872948, 0.034), (2954456800, 0.03), (2377424484, 0.024), (2903764084, 0.019), (2921101790, 0.002)]",7,alexander wachtel


## Survey evaluation

In [27]:
from survey_expert_list import survey_expert_list
survey_test_data = ("survey test data", [{"keyword": topic, "paper_ids":experts} for topic, experts  in survey_expert_list.items()])

In [38]:
survey_results = evaluate_models([("Best expert voting model",best_voting_model,None)], [survey_test_data],1, metrics)

In [39]:
calculate_metrics(best_voting_model, survey_test_data[1], metrics)

,query,r@5,r@10,r@20
0,humanoid robot,0.666667,0.666667,1.000000
1,natural language processing,0.500000,0.666667,0.833333
2,internet of things,0.666667,0.666667,0.666667
3,evolutionary algorithm,1.000000,1.000000,1.000000


In [35]:
survey_results

survey test data                                  
                                      r@5         r@10          r@20       
                                      avg    var   avg    var    avg    var
Best expert voting model            0.708  0.044  0.75  0.028  0.875  0.025

In [41]:
print(author_ranking_to_latex(survey_results))

Best expert voting model & 0.708 & 0.750 & 0.875\\



In [23]:
best_voting_model.get_ranking("fuel cells", author_info).head(10)

,id,score,relevant_docs,#rel docs,name
0,1551550943,0.791,"[(146892559, 1.164), (2259390525, 0.367), (1512170180, 0.271), (2038691916, 0.104), (2325229957, 0.039), (2023463225, 0.026), (1503474360, 0.021), (2094789278, 0.007), (1970377605, 0.007), (2154295492, 0.006), (2129831810, 0.004), (2961682653, 0.003), (1976711823, 0.003), (2810103608, 0.003), (2803765856, 0.003), (1983754877, 0.003), (2113254436, 0.001), (2277757309, 0.0), (2070839090, 0.0), (2807223683, 0.0), (2586783924, 0.0), (1992754411, 0.0), (1966258098, 0.0), (2047712612, 0.0), (20668...",27,bradley p ladewig
2,2145260813,0.366,"[(2571265703, 0.23), (2065924407, 0.05), (1979164931, 0.031), (2789047057, 0.021), (2713551381, 0.015), (2086158641, 0.012), (2041682082, 0.011), (2064096673, 0.009), (2914431538, 0.005), (1981219625, 0.003), (2888107718, 0.001), (2751472405, 0.001), (2219374902, 0.0), (2285567507, 0.0), (2982260244, 0.0), (2981981258, 0.0), (2574686799, 0.0), (2271915128, 0.0), (2885221775, 0.0), (2978567381, 0.0), (2912240687, 0.0), (2054805466, 0.0), (284997139, 0.0), (2611363107, 0.0), (2255394328, 0.0),...",100,maximilian fichtner
1,1973792470,0.232,"[(1495805924, 0.573), (2255589006, 0.008), (2308920088, 0.002), (2992000058, 0.001), (2947103518, 0.0), (1992131983, 0.0), (1555878363, 0.0), (2029468296, 0.0), (2046871861, 0.0), (2000238401, 0.0), (2023880326, 0.0), (1984933003, 0.0), (2970543498, 0.0), (1980577508, 0.0), (2028974511, 0.0), (1987335962, 0.0), (2047946942, 0.0), (2754262456, 0.0), (2137011090, 0.0), (2212601263, 0.0), (2054885791, 0.0), (2093278617, 0.0), (2100234056, 0.0), (2122301258, 0.0), (1680945277, 0.0), (2048497532,...",70,c heske
8,867224093,0.218,"[(54182603, 0.085), (2185229993, 0.024), (2323597074, 0.016), (1970160332, 0.013), (2039238970, 0.013), (2126508225, 0.011), (2153620342, 0.01), (2142333515, 0.009), (2138967413, 0.007), (2004312148, 0.007), (2516647448, 0.006), (2084755666, 0.006), (2189052352, 0.005), (2139079838, 0.005), (2166822665, 0.004), (2066099168, 0.004), (2184074290, 0.004), (2080031812, 0.004), (2047460956, 0.003), (1974460965, 0.003), (2891192286, 0.003), (2188614107, 0.003), (2953361946, 0.002), (2908421526, 0....",67,olaf deutschmann
4,271445011,0.167,"[(2464240077, 0.115), (2158655454, 0.036), (2053018838, 0.019), (2999315498, 0.007), (2465367442, 0.006), (2068302422, 0.005), (1984230077, 0.005), (2104236514, 0.005), (2514926759, 0.005), (1997573525, 0.003), (2058755723, 0.002), (2123675093, 0.001), (2395376770, 0.001), (2261960623, 0.001), (2462583094, 0.001), (2466872089, 0.001), (2585425579, 0.0), (2281484173, 0.0), (2021588739, 0.0), (2620013420, 0.0), (2021260971, 0.0), (2171103484, 0.0), (2147730030, 0.0), (2060446801, 0.0), (241698...",29,r reimert
9,1909062222,0.143,"[(1975675698, 0.035), (2330684406, 0.033), (2387797090, 0.031), (2029261022, 0.02), (2031255796, 0.017), (2586018673, 0.014), (2117842934, 0.012), (1967351109, 0.011), (2508686508, 0.01), (2467621168, 0.007), (2760901006, 0.006), (2606654015, 0.004), (2773905847, 0.002), (2996006733, 0.0), (2901719466, 0.0)]",15,roswitha zeis
11,1739862885,0.134,"[(1968357813, 0.059), (2087618759, 0.028), (2036534310, 0.023), (2082439773, 0.012), (2901451920, 0.01), (2062454621, 0.006), (2922291321, 0.005), (2044684288, 0.004), (1990998245, 0.003), (2754203924, 0.002), (2023652754, 0.001), (2616909392, 0.001), (2067343506, 0.001), (2977523176, 0.001), (2056050167, 0.001), (2969381489, 0.001), (2810531016, 0.001), (2139510921, 0.001), (2742393163, 0.001), (2035106021, 0.001), (2900600478, 0.0), (2035603810, 0.0), (1971488541, 0.0), (1518572017, 0.0), ...",58,roland dittmeyer
6,2591427225,0.129,"[(2041601875, 0.13), (2805764891, 0.021), (2093644384, 0.01), (1993625348, 0.009), (2008321772, 0.009), (2087972022, 0.007), (2035053446, 0.005), (2072088124, 0.005), (2079740907, 0.004), (1992485436, 0.003), (2754203924, 0.002), (2946048241, 0.002), (2330531691, 0.001), (2616909392, 0.001), (2067343506, 0.001), (2913239356, 0.001), (2086722104, 0.0), (2900

In [32]:
calculate_metrics(best_voting_model, kompetenzpool_test_data[1], metrics)

,query,r@5,r@10,r@20
0,Internet of Things,0.666667,0.666667,0.666667
1,blockchain,0.000000,0.500000,0.500000
2,natural language processing,0.000000,0.500000,1.000000
3,autonomous driving,0.000000,0.500000,1.000000
4,humanoid robot,0.125000,0.125000,0.750000
5,evolutionary algorithm,1.000000,1.000000,1.000000
6,remote sensing,0.250000,0.500000,0.500000
7,cellular automaton,1.000000,1.000000,1.000000
8,electronic voting,0.000000,1.000000,1.000000
9,kalman filter,0.666667,0.666667,1.000000


In [416]:
# only use average of first 5 papers and multiply by 5
# only count co-authors of relevant papers


document_score_modifiers = [
#     ExponentialModifier(),
    ReciprocalRankModifier(True),
    PaperAgeModifier(paper_info, 2, 0.6),
#     CitationCountModifier(paper_info, 0.01)
]
aggregation = SummationAggregation(paper_info,10)#AuthorOrderAggregation(paper_info, 0.4)
author_score_modifiers = [
#     PaperCountModifier(paper_info),
    CoAuthorModifier(paper_info,0.015),
    LastKitPaperModifier(author_info, 3, 0.1),
    KitPaperRatioModifier(author_info, 0.01),
]
expert_voting = ExpertVoting(document_score_modifiers, aggregation, author_score_modifiers, model)

In [96]:
def add_correct(result, ground_truth):
    result["correct"] = result["id"].isin(ground_truth["paper_ids"])
    return result

In [150]:
"2498022428" in author_info

True

In [372]:
sample = test_data[1][7]

In [373]:
calculate_metrics(expert_voting, [sample], default_metrics)

,query,p@5,p@10,p@20,R-prec,mAP,bpref
0,action recognition,0.6,0.833333,0.833333,0.666667,0.662037,0.638889


In [444]:
expert_voting.get_ranking("climate change", author_info).head(10)

,id,score,relevant_docs,#rel docs,name
1,2116430738,2.213,"[(2895907029, 0.975), (1923707467, 0.553), (2981747039, 0.378), (2162421321, 0.172), (2625635056, 0.139), (2803060350, 0.125), (2282323584, 0.101), (2083885861, 0.088), (2095989966, 0.081), (2391010793, 0.071), (2789395858, 0.059), (2118921617, 0.058), (2035962295, 0.054), (1949835922, 0.047), (2890569313, 0.038), (2788113031, 0.034), (2615953471, 0.032), (2133536153, 0.03), (2001322891, 0.027), (2791837888, 0.024), (2154981955, 0.022), (2120532631, 0.021), (2099536556, 0.019), (2604283558, ...",210,almut arneth
0,2054842659,1.029,"[(2157538991, 1.629), (2053054758, 0.396), (2981747039, 0.378), (2148467068, 0.114), (768158134, 0.066), (2034228507, 0.036), (2060201736, 0.035), (1717199148, 0.025), (2791837888, 0.024), (2168515722, 0.022), (2036492581, 0.02), (2090317461, 0.018), (2542613894, 0.018), (2018292862, 0.018), (2055258200, 0.016), (2593643591, 0.016), (2061507888, 0.016), (2005470817, 0.014), (2579935032, 0.013), (2997129879, 0.012), (2755975859, 0.012), (2766346838, 0.011), (1991725666, 0.01), (2406049275, 0....",138,mark rounsevell
2,2588254444,0.514,"[(2981747039, 0.378), (2625635056, 0.139), (2789395858, 0.059), (2615953471, 0.032), (2791837888, 0.024), (2952896657, 0.01), (2467022070, 0.005), (2766601134, 0.001), (2795657023, 0.001), (2978485176, 0.001), (2968516580, 0.0)]",11,peter anthoni
3,2126441205,0.456,"[(158353950, 0.318), (29127752, 0.06), (1585299485, 0.039), (2003497260, 0.034), (2147659164, 0.019), (2073965075, 0.019), (2052267546, 0.015), (1977020983, 0.012), (2052583168, 0.008), (2041275546, 0.007), (2021416786, 0.007), (2087655325, 0.006), (2022996047, 0.005), (2902715277, 0.005), (2807585951, 0.005), (2898499686, 0.004), (2127375643, 0.004), (2089592562, 0.004), (1975273625, 0.003), (2121818186, 0.003), (2045931643, 0.003), (2071910167, 0.003), (2770755223, 0.003), (2033033853, 0.0...",84,joaquim g pinto
5,2104736921,0.371,"[(2981747039, 0.378), (2791837888, 0.024), (1719633689, 0.009), (2196451352, 0.006), (2922146099, 0.003), (2999644459, 0.002), (2566857028, 0.002), (2301696792, 0.001), (2734683486, 0.001), (2130007629, 0.001), (2968516580, 0.0), (2205022211, 0.0), (2606687499, 0.0), (2596816404, 0.0)]",14,sam rabin
4,2609496848,0.341,"[(2141609924, 0.197), (2280126850, 0.158), (2905251100, 0.033), (2128071883, 0.028), (2899012876, 0.027), (2099017271, 0.005), (2153306908, 0.002), (2086373815, 0.002), (2750704026, 0.002), (2089208284, 0.002), (2148065061, 0.002), (2088978228, 0.002), (2136717810, 0.002), (2112643492, 0.002), (2335452896, 0.001), (2085941609, 0.001), (1819626098, 0.001), (2047970156, 0.001), (2120718868, 0.001), (2142967622, 0.001), (2607928283, 0.001), (2132991176, 0.0), (2133081671, 0.0), (1984177205, 0.0...",56,h p schmid
10,2113369698,0.289,"[(2141609924, 0.197), (2905251100, 0.033), (2899012876, 0.027), (2517610868, 0.019), (2971801865, 0.017), (2358327662, 0.016), (2081560697, 0.009), (2739350169, 0.009), (2289151841, 0.008), (2944191255, 0.007), (2912131008, 0.007), (2924400300, 0.006), (2329974787, 0.005), (2614047786, 0.004), (2149869919, 0.004), (2921034716, 0.004), (2766688267, 0.003), (2972475990, 0.003), (2086373815, 0.002), (2014155683, 0.002), (2056007219, 0.002), (2963144957, 0.002), (2472719015, 0.002), (2566291100,...",92,ralf kiese
6,2168384738,0.273,"[(158353950, 0.318), (2460625541, 0.015), (2031210658, 0.01), (1974446809, 0.008), (2234129322, 0.008), (2587987274, 0.004), (2605781041, 0.003), (1975020664, 0.003), (2110545604, 0.003), (2106593015, 0.002), (2082323488, 0.002), (2048185203, 0.002), (2903730579, 0.002), (1994397639, 0.002), (2266091352, 0.002), (161862615, 0.001), (2605990029, 0.001), (2889909242, 0.001), (18643346, 0.001), (1552753261, 0.001), (2625047403, 0.001), (2735064053, 0.001), (2559092926, 0.001), (2509620309, 0.00...",58,andreas h fink
7,2028622345,0.271,"[(2141609924, 0.197), (2391010793, 0.071), (2899012876, 0.027), (2517610868, 0.019), (2971801865, 0.017), (2358327662, 0.016

In [421]:
r = add_correct(expert_voting.get_ranking(sample["keyword"], author_info), sample).reset_index(drop=True)
# r[r.id == 2263767454]
r.head(40)

,id,score,relevant_docs,#rel docs,name,correct
0,332721774,1.186,"[(2400236354, 0.716), (2941505483, 0.412), (1995939942, 0.213), (2911959490, 0.107), (2398968536, 0.077), (2941712317, 0.073), (1977646036, 0.066), (2896950454, 0.059), (1853352468, 0.043), (2028158532, 0.034), (2990685265, 0.025), (2124218221, 0.017), (1602217685, 0.013), (2139880049, 0.012), (1502297357, 0.011), (2123132313, 0.01), (134419874, 0.009), (2058666174, 0.007), (1517562115, 0.006), (2160958403, 0.005), (1147231896, 0.005), (2986811360, 0.004), (2168821116, 0.004), (1498185956, 0...",188,rainer stiefelhagen,True
1,1047662447,0.356,"[(2981773282, 0.163), (1487977235, 0.161), (2510756328, 0.03), (2152897124, 0.025), (2571411619, 0.023), (2004707451, 0.009), (2106321383, 0.008), (2139815705, 0.008), (2219532056, 0.008), (2085164197, 0.006), (1492952184, 0.005), (2029013291, 0.005), (2910448276, 0.003), (2792815601, 0.003), (2132602743, 0.003), (2541171006, 0.003), (2963410356, 0.003), (2066407408, 0.003), (2107987248, 0.003), (2140434853, 0.003), (2601766769, 0.002), (2122467384, 0.002), (1995034212, 0.002), (2132905014, ...",191,tamim asfour,False
2,2161631798,0.174,"[(2981773282, 0.163), (2571411619, 0.023), (2004707451, 0.009), (2219532056, 0.008), (1492952184, 0.005), (2780774970, 0.001), (2912479198, 0.001), (2912701897, 0.001), (2782973409, 0.0), (1967407878, 0.0), (2783028038, 0.0), (2890646997, 0.0), (2789941543, 0.0), (2034173299, 0.0), (2570226614, 0.0), (2555794269, 0.0), (2963573295, 0.0), (2767706754, 0.0), (2034131903, 0.0)]",19,mirko wachter,False
3,291719226,0.143,"[(1995939942, 0.213), (2067175278, 0.001), (1568641926, 0.0), (2060662362, 0.0), (2952749830, 0.0), (2311221615, 0.0), (2005285092, 0.0), (2981820411, 0.0), (2929483642, 0.0), (2902734834, 0.0), (2920998074, 0.0), (2997646412, 0.0), (2912709362, 0.0)]",13,ziad alhalah,True
4,2782551808,0.135,"[(2981773282, 0.163), (2783028038, 0.0)]",2,christian r g dreher,True
5,13246170,0.096,"[(1977646036, 0.066), (2124218221, 0.017), (1484536156, 0.016), (2103725923, 0.013), (1579386697, 0.001), (2148989010, 0.001), (1982263908, 0.0), (1549156782, 0.0), (2145288592, 0.0), (2576651838, 0.0), (1519254153, 0.0), (2395522772, 0.0), (2041186355, 0.0), (2131979035, 0.0), (202215066, 0.0), (366649198, 0.0), (2264164804, 0.0), (121144273, 0.0), (2011202541, 0.0), (2588920783, 0.0), (46606381, 0.0), (2402929600, 0.0), (120694877, 0.0), (2102626948, 0.0), (2148247654, 0.0), (2589076805, 0...",67,uwe d hanebeck,False
6,2989702969,0.095,"[(2941505483, 0.412), (2941712317, 0.073), (2962902099, 0.0)]",3,vivek sharma,True
7,2562646779,0.088,"[(2911959490, 0.107), (1853352468, 0.043), (2058761328, 0.002), (1997513865, 0.001), (2979717350, 0.0), (2188259590, 0.0), (2737146145, 0.0), (2181567432, 0.0), (2768577426, 0.0), (214684151, 0.0), (2294133535, 0.0), (2613930560, 0.0), (2396974517, 0.0), (1531200232, 0.0)]",14,manuel martinez,False
8,2105785781,0.085,"[(58335600, 0.126), (1511860436, 0.001), (2069395995, 0.001), (1599184066, 0.001), (1984342400, 0.001), (2112290235, 0.001), (2074782959, 0.001), (1976015523, 0.001), (1514485291, 0.001), (2295504855, 0.0), (1970126562, 0.0), (1969273236, 0.0), (2407682997, 0.0), (2001121503, 0.0), (2080674188, 0.0), (63010170, 0.0), (1989138950, 0.0), (2001271136, 0.0), (1965043025, 0.0), (2397106775, 0.0), (2091337500, 0.0), (1912178518, 0.0), (2007333865, 0.0), (1000921648, 0.0), (2055794562, 0.0)]",25,felix putze,False
9,2913758523,0.076,"[(2911959490, 0.107), (2986811360, 0.004), (2981820411, 0.0), (2966369713, 0.0), (2920998074, 0.0), (2737146145, 0.0), (2912709362, 0.0)]",7,monica haurilet,False


In [ ]:
model.weight = 0.02

In [ ]:
r = model.get_ranking("natural language processing")
# r[r.id == 2596026555]
r

In [ ]:
model.get_ranking("blockchain")

In [ ]:
evaluate_model(expert_voting, [test_data])[('manual test data', 'mAP', 'avg')]

In [ ]:
evaluate_model(expert_voting, [test_data])[('manual test data', 'mAP', 'avg')]

In [417]:
%%time
calculate_metrics(expert_voting, test_data[1], default_metrics)

CPU times: user 28.4 s, sys: 37.8 s, total: 1min 6s
Wall time: 12.3 s


,query,p@5,p@10,p@20,R-prec,mAP,bpref
0,internet of things,1.00,0.600000,0.600000,0.600000,0.635488,0.570000
1,blockchain,1.00,1.000000,0.846154,0.846154,0.896961,0.852071
2,natural language processing,0.80,0.700000,0.615385,0.538462,0.586041,0.526627
3,autonomous driving,1.00,0.600000,0.647059,0.647059,0.606622,0.577855
4,humanoid robot,0.80,0.800000,0.882353,0.823529,0.791908,0.799308
5,evolutionary algorithm,0.80,0.714286,0.857143,0.571429,0.782115,0.693878
6,remote sensing,1.00,0.800000,0.550000,0.583333,0.614922,0.539931
7,action recognition,0.60,0.666667,0.833333,0.500000,0.520996,0.472222
8,cellular automaton,1.00,1.000000,1.000000,0.666667,0.866667,0.777778
9,electronic voting,0.75,1.000000,1.000000,0.750000,0.787500,0.687500


In [310]:
%%time
calculate_metrics(expert_voting, test_data[1], default_metrics)

CPU times: user 27.8 s, sys: 38.7 s, total: 1min 6s
Wall time: 12.2 s


,query,p@5,p@10,p@20,R-prec,mAP,bpref
0,internet of things,1.000000,0.600000,0.700000,0.600000,0.687662,0.640000
1,blockchain,0.800000,0.900000,0.923077,0.846154,0.842574,0.857988
2,natural language processing,1.000000,0.700000,0.615385,0.538462,0.644651,0.573964
3,autonomous driving,1.000000,0.800000,0.705882,0.705882,0.679170,0.653979
4,humanoid robot,0.800000,0.800000,0.882353,0.764706,0.792342,0.795848
5,evolutionary algorithm,0.800000,0.857143,1.000000,0.714286,0.712358,0.693878
6,remote sensing,1.000000,0.900000,0.650000,0.541667,0.672975,0.623264
7,action recognition,0.600000,0.833333,0.833333,0.666667,0.673611,0.666667
8,cellular automaton,0.666667,1.000000,1.000000,0.666667,0.833333,0.666667
9,electronic voting,0.750000,1.000000,1.000000,0.750000,0.916667,0.875000


In [272]:
calculate_metrics(expert_voting, test_data[1], default_metrics)

,query,p@5,p@10,p@20,R-prec,mAP,bpref
0,internet of things,1.000000,0.600000,0.700000,0.600000,0.724406,0.660000
1,blockchain,1.000000,0.800000,0.769231,0.692308,0.743956,0.698225
2,natural language processing,1.000000,0.700000,0.692308,0.615385,0.681762,0.639053
3,autonomous driving,0.800000,0.800000,0.470588,0.470588,0.487860,0.449827
4,humanoid robot,0.800000,0.900000,0.764706,0.764706,0.817194,0.826990
5,evolutionary algorithm,0.600000,0.857143,1.000000,0.714286,0.645493,0.612245
6,remote sensing,1.000000,0.900000,0.700000,0.708333,0.706039,0.680556
7,action recognition,0.400000,0.333333,0.500000,0.333333,0.321740,0.305556
8,cellular automaton,0.666667,1.000000,1.000000,0.666667,0.809524,0.666667
9,electronic voting,1.000000,1.000000,1.000000,0.750000,0.950000,0.937500


In [ ]:
evaluate_model(expert_voting, [test_data])#[('manual test data', 'mAP', 'avg')]

In [ ]:
evaluate_model(expert_voting, [test_data])#[('manual test data', 'mAP', 'avg')]

In [ ]:
evaluate_model(expert_voting, [test_data])#[('manual test data', 'mAP', 'avg')]

In [ ]:
ranking = model.get_ranking("natural language processing")

In [ ]:
scored_documents = [ScoredDocument(doc[1],doc[2]) for doc in ranking.to_records()]

In [ ]:
pd.DataFrame(expert_voting.vote(scored_documents))

In [ ]:
a.to_records()[1]